In [38]:
import stealth_requests as requests
import requests as r
from bs4 import BeautifulSoup
from markdownify import markdownify as md
from dotenv import load_dotenv
import os
import json
from tqdm import tqdm
from time import sleep
load_dotenv()

True

In [39]:
with open('data/urls.txt', 'r') as f:
    page_urls = [line.strip() for line in f if line.strip()]

In [40]:
def scrape_content(url):
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, 'html.parser')

    # Find the 'div' with class 'content-wrap'
    content_wrap_div = soup.find('div', class_='content-wrap')

    # Get the HTML content of this div
    if content_wrap_div:
        content_wrap_html = str(content_wrap_div)
        md_content = md(content_wrap_html)
        #print(md_content)
        return md_content
    else:
        print("Div with class 'content-wrap' not found.")
        return None


In [41]:
import base64

pbar = tqdm(page_urls)

markdown_dicts = []
for url in pbar:
    pbar.set_description(f"Processing {url}")
    url = url
    content = scrape_content(url)
    if content:
        markdown_dicts.append({
            "url": url,
            "content": content
        })
    sleep(0.5)  # To avoid overwhelming the server


Processing https://summer.skku.edu/summer/index.do:   0%|          | 0/30 [00:00<?, ?it/s]

Div with class 'content-wrap' not found.


Processing https://summer.skku.edu/summer/registration/calendar.do:  13%|█▎        | 4/30 [00:03<00:19,  1.36it/s] 

Div with class 'content-wrap' not found.


Processing https://summer.skku.edu/summer/registration/application2.do:  47%|████▋     | 14/30 [00:10<00:11,  1.43it/s]

Div with class 'content-wrap' not found.


Processing https://summer.skku.edu/summer/registration/insurance.do: 100%|██████████| 30/30 [00:21<00:00,  1.38it/s]   


In [35]:
import base64

markdown_dicts = []
for url in page_urls:
    print(f"Scraping {url}...")
    url = url
    content = scrape_content(url)
    markdown_bytes = content.encode('utf-8')
    base64_bytes = base64.b64encode(markdown_bytes)
    if content:
        markdown_dicts.append({
            "url": url,
            "content": content,
            "base64": base64_bytes.decode('utf-8')
        })


Scraping https://summer.skku.edu/summer/index.do...
Div with class 'content-wrap' not found.


AttributeError: 'NoneType' object has no attribute 'encode'

In [ ]:
api_url = "http://localhost:8080/"
api_key = os.getenv("OPEN_WEBUI_API_KEY")
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json",
    'Accept': 'application/json'
}

In [ ]:
knowledge_base_data = {
    "name": "ISS Info",
    "description": "Knowledge Base with information about International Summer Semester",
    "data": {},
    "access_control": {}
}

# The api_url is defined in cell 4
create_kb_url = f"{api_url}api/v1/knowledge/create"
response = r.post(create_kb_url, headers=headers, json=knowledge_base_data)
if response.status_code == 200:
    kb_creation_response = response.json()
    kb_creation_id = kb_creation_response.get("id")
    print(kb_creation_response)
else:
    print(f"Failed to create knowledge base: {response.status_code} - {response.text}")

In [ ]:
kb_creation_id

In [ ]:
api_add_file_url = "/api/v1/files/"

In [ ]:
def add_file_to_kb(source_url, base64_content):
    api_add_file_url = "/api/v1/files/"
    files = {'file': base64_content}
    response = r.post(f"{api_url}{api_add_file_url}", headers=headers, files=files)
    return response

In [ ]:
add_file_to_kb